In [29]:
# Library Import
import os 
import pandas as pd
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

pd.options.mode.use_inf_as_na = True

In [30]:
# Parameters

Host = "https://strategenics.atlassian.net/"
Username = "mike.burns@strategenics.com.au"
Password = "ATATT3xFfGF0wcJfZvitiiWKYW_9UCdV2r1dWJE6O8a7ZxFRQUU7FVnBkdYlxNx9PcEn_tbHM-tdxz4DZedCozWuNUSI46DcZDPMKpXq0QDgiWomXTEkzDjVZfVa4oRjADfcbgebgME5skwsc5Ue4doiLA1OpiIyxzP1gzXAzNcfYSniUnKKoqI=2FD01AC0"
ValidProjectCategories = ["'Customer Delivery Projects'"]

In [31]:
# Functions
import base64
import pandas as pd
import re 
import requests
from functools import reduce

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        

        response = requests.post(Host + url, headers = headers, json = defaultContents)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes):
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = fnExpandColumn(df, "projects", None)
    df = df[["issuetypes"]]
    df = df.explode("issuetypes")
    df = fnExpandColumn(df, "issuetypes", None)
    df = df[["fields"]]
    df = fnExpandColumn(df, "fields", None)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    df = pd.DataFrame(values)
    df = df.sort_values("fieldId")
    return df 

def fnGetValidProjectKeys():
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    df = fnAPI(ApiCall)
    df = fnExpandColumn(df, "projectCategory")
    df = df.loc[df['projectCategory name'].isin(ValidProjectCategories) | ("'" + df['projectCategory name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnSentenceCase(s):
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]).replace("0", "")

def fnAPI(webRequestDelegate, startAt = 0): 
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df
    
def fnExpandColumn(df:pd.DataFrame, colName:str, prefix:str = "Prefix "):
    if prefix == "Prefix ":
        prefix = colName + " "
    if prefix == None: 
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x))], axis=1)
    else:
        df = pd.concat([df.drop(colName, axis=1), df[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))], axis=1)
    return df

In [32]:
# Projects

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/project"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "expand": "projectKeys"
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["avatarUrls", "expand", "self", "entityId", "uuid"], axis=1)
df = fnExpandColumn(df, "projectCategory")
df = df.drop(["projectCategory self"], axis=1)
df["Is Valid"] = (
    df['projectCategory name'].isin(ValidProjectCategories) | ("'" + df['projectCategory name'] + "'").isin(ValidProjectCategories)
)
df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')
df["projectCategory id"] = df["projectCategory id"].astype('Int64')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Id                                       Int64
Key                             string[python]
Name                            string[python]
Project Keys                            object
Project Type Key                string[python]
Simplified                             boolean
Style                           string[python]
Is Private                             boolean
Properties                              object
Archived                               boolean
Archived Date                   string[python]
Archived By                             object
Project Category Id                      Int64
Project Category Name           string[python]
Project Category Description    string[python]
Project Category                         Int64
Is Valid                               boolean
dtype: object

,Id,Key,Name,Project Keys,Project Type Key,Simplified,Style,Is Private,Properties,Archived,Archived Date,Archived By,Project Category Id,Project Category Name,Project Category Description,Project Category,Is Valid
0,13098,AHPRA,AHPRA,[AHPRA],service_desk,False,classic,True,{},<NA>,<NA>,NaN,10106,Customer Delivery Projects (Atlassian),Atlassian Customer Delivery Projects,<NA>,False
1,13177,AAS,AHPRA Application Support,[AAS],service_desk,False,classic,True,{},<NA>,<NA>,NaN,10106,Customer Delivery Projects (Atlassian),Atlassian Customer Delivery Projects,<NA>,False
2,13091,AWS,Amazon Web Services,[AWS],business,False,classic,False,{},<NA>,<NA>,NaN,10102,Internal Projects,Internal Strategenics projects,<NA>,False
3,13173,ANGT,Anglicare,"[ANGT, A1]",service_desk,False,classic,True,{},<NA>,<NA>,NaN,10106,Customer Delivery Projects (Atlassian),Atlassian Customer Delivery Projects,<NA>,False
4,13101,PD,Archived Project Delivery,[PD],software,False,classic,False,{},<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,13043,PP,ZZ - PinPoint (Pest Monitoring),[PP],software,False,classic,False,{},True,2022-07-27T16:09:13.865+1000,{'self': 'https://strategenics.atlassian.net/r...,10104,Product Development,A project for a product developed by Strategenics,<NA>,False
78,13005,SG738,ZZ - QGCIO Cybersecurity (SG738),"[SG738, QC]",software,False,classic,False,{},True,2022-07-27T16:09:27.771+1000,{'self': 'https://strategenics.atlassian.net/r...,10101,Customer Delivery Projects,A project set up to manage deliverables for a ...,<NA>,True
79,13058,QTSIB,ZZ - Queensland Treasury - Strategic Informati...,[QTSIB],software,False,classic,False,{},True,2022-07-27T16:09:33.217+1000,{'self': 'https://strategenics.atlassian.net/r...,10101,Customer Delivery Projects,A project set up to manage deliverables for a ...,<NA>,True
80,13051,SE,ZZ - Softlink Education,"[SE, STIH]",software,False,classic,False,{},True,2022-07-27T16:09:38.499+1000,{'self': 'https://strategenics.atlassian.net/r...,10101,Customer Delivery Projects,A project set up to manage deliverables for a ...,<NA>,True


In [33]:
# Project Categories

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/projectCategory"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)
df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')

list_row = [0, "No Category", "No Category"]
df.loc[len(df)] = list_row
df = df.sort_values('id')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Id              Int64
Description    object
Name           object
dtype: object

,Id,Description,Name
9,0,No Category,No Category
7,10000,,Software Development
8,10100,,Task Tracking
0,10101,A project set up to manage deliverables for a ...,Customer Delivery Projects
2,10102,Internal Strategenics projects,Internal Projects
6,10103,A project to support internal Strategenics Res...,Research & Development Projects
5,10104,A project for a product developed by Strategenics,Product Development
4,10105,Projects for ongoing managed services,Managed Services
1,10106,Atlassian Customer Delivery Projects,Customer Delivery Projects (Atlassian)
3,10107,,Internal Projects (Atlassian)


In [34]:
# Issue Types
from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/issuetype"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)
df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')

df = df.sort_values('name')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Id                            Int64
Description          string[python]
Icon Url             string[python]
Name                 string[python]
Untranslated Name    string[python]
Subtask                     boolean
Avatar Id                     Int64
Hierarchy Level               Int64
Scope                        object
dtype: object

,Id,Description,Icon Url,Name,Untranslated Name,Subtask,Avatar Id,Hierarchy Level,Scope
61,10104,For new system accounts or passwords. Created ...,https://strategenics.atlassian.net/servicedesk...,Access,Access,False,<NA>,0,NaN
50,10984,Have a question? Submit it here.,https://strategenics.atlassian.net/rest/api/2/...,Ask a question,Ask a question,False,13390,0,NaN
7,1,A problem which impairs or prevents the functi...,https://strategenics.atlassian.net/rest/api/2/...,Bug,Bug,False,10303,0,NaN
3,10902,,https://strategenics.atlassian.net/rest/api/2/...,Bug*,Bug*,True,10303,-1,NaN
59,10102,For system upgrades or alterations. Created by...,https://strategenics.atlassian.net/servicedesk...,Change,Change,False,<NA>,0,NaN
...,...,...,...,...,...,...,...,...,...
35,10976,Created by Jira Service Management.,https://strategenics.atlassian.net/rest/api/2/...,[System] Change,[System] Change,False,13352,0,NaN
32,10973,For system outages or incidents. Created by Ji...,https://strategenics.atlassian.net/rest/api/2/...,[System] Incident,[System] Incident,False,13349,0,NaN
36,10977,Track underlying causes of incidents. Created ...,https://strategenics.atlassian.net/rest/api/2/...,[System] Problem,[System] Problem,False,13353,0,NaN
33,10974,A request that follows ITSM workflows.,https://strategenics.atlassian.net/rest/api/2/...,[System] Service request,[System] Service request,False,13350,0,NaN


In [35]:
# Project Types

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/project/type"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.convert_dtypes().infer_objects()
df = df.sort_values('key')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Key                     string[python]
Formatted Key           string[python]
Description I18n Key    string[python]
Icon                    string[python]
Color                   string[python]
dtype: object

,Key,Formatted Key,Description I18n Key,Icon,Color
3,business,Business,jira.project.type.business.description,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#1D8832
0,product_discovery,Product Discovery,jira.project.type.polaris.description,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#F5A623
2,service_desk,Service Desk,jira.project.type.servicedesk.description.jsm,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#67AB49
1,software,Software,jira.project.type.software.description,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#F5A623


In [36]:
# Resolutions

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/resolution"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)
df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')
df = df.sort_values('name')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Id                      Int64
Description    string[python]
Name           string[python]
dtype: object

,Id,Description,Name
2,10301,,Cancelled
7,10302,,Cannot Fix
5,5,"All attempts at reproducing this issue failed,...",Cannot Reproduce
11,10200,This issue was not approved.,Declined
0,10000,,Done
3,3,The problem is a duplicate of an existing issue.,Duplicate
1,1,A fix for this issue is checked into the tree ...,Fixed
12,10303,,Hardware failure
4,4,The problem is not completely described.,Incomplete
6,10201,The problem has a documented root cause and a ...,Known Error


In [37]:
# Status

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/status"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self", "scope"], axis=1)
df = fnExpandColumn(df, "statusCategory")

df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')
df = df.sort_values('name')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Description                   string[python]
Icon Url                      string[python]
Name                          string[python]
Untranslated Name             string[python]
Id                                     Int64
Status Category Self          string[python]
Status Category Id                     Int64
Status Category Key           string[python]
Status Category Color Name    string[python]
Status Category Name          string[python]
dtype: object

,Description,Icon Url,Name,Untranslated Name,Id,Status Category Self,Status Category Id,Status Category Key,Status Category Color Name,Status Category Name
85,,https://strategenics.atlassian.net/images/icon...,Assigned,Assigned,11955,https://strategenics.atlassian.net/rest/api/la...,2,new,blue-gray,To Do
67,This was auto-generated by Jira Service Manage...,https://strategenics.atlassian.net/images/icon...,Authorize,Authorize,11929,https://strategenics.atlassian.net/rest/api/la...,4,indeterminate,yellow,In Progress
60,This was auto-generated by Jira Service Desk d...,https://strategenics.atlassian.net/images/icon...,Awaiting implementation,Awaiting implementation,11847,https://strategenics.atlassian.net/rest/api/la...,4,indeterminate,yellow,In Progress
11,,https://strategenics.atlassian.net/images/icon...,Backlog,Backlog,10400,https://strategenics.atlassian.net/rest/api/la...,2,new,blue-gray,To Do
48,,https://strategenics.atlassian.net/images/icon...,Beta/UAT,Beta/UAT,11810,https://strategenics.atlassian.net/rest/api/la...,4,indeterminate,yellow,In Progress
...,...,...,...,...,...,...,...,...,...,...
54,,https://strategenics.atlassian.net/images/icon...,Waiting for Bug Fix,Waiting for Bug Fix,11829,https://strategenics.atlassian.net/rest/api/la...,2,new,blue-gray,To Do
7,This was auto-generated by JIRA Service Desk d...,https://strategenics.atlassian.net/images/icon...,Waiting for Customer,Waiting for Customer,10001,https://strategenics.atlassian.net/rest/api/la...,2,new,blue-gray,To Do
6,This was auto-generated by JIRA Service Desk d...,https://strategenics.atlassian.net/images/icon...,Waiting for Support,Waiting for Support,10000,https://strategenics.atlassian.net/rest/api/la...,4,indeterminate,yellow,In Progress
58,This was auto-generated by Jira Service Desk d...,https://strategenics.atlassian.net/images/icon...,Waiting for approval,Waiting for approval,11844,https://strategenics.atlassian.net/rest/api/la...,2,new,blue-gray,To Do


In [38]:
# Status Categories

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/statuscategory"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')
df = df.sort_values('id')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Id                     Int64
Key           string[python]
Color Name    string[python]
Name          string[python]
dtype: object

,Id,Key,Color Name,Name
0,1,undefined,medium-gray,No Category
1,2,new,blue-gray,To Do
3,3,done,green,Done
2,4,indeterminate,yellow,In Progress


In [42]:
# Users

from IPython.display import display
import requests
import pandas as pd

def ApiCall(startAt) :
    url = "/rest/api/latest/users/search"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "maxResults": "1000"
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = pd.concat([fnAPI(ApiCall, x * 1000) for x in range(0, 10)])
df = df.drop(["self"], axis=1)

df = fnExpandColumn(df, "avatarUrls")
df = df.drop(["avatarUrls 16x16", "avatarUrls 24x24", "avatarUrls 32x32"], axis=1)
df = df.rename(columns={"avatarUrls 48x48": "avatarUrl"})

df = df.convert_dtypes().infer_objects()
df = df.sort_values('displayName')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Account Id       string[python]
Account Type     string[python]
Email Address    string[python]
Display Name     string[python]
Active                  boolean
Time Zone        string[python]
Locale           string[python]
Avatar Url       string[python]
dtype: object

,Account Id,Account Type,Email Address,Display Name,Active,Time Zone,Locale,Avatar Url
391,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:5c80c3...,customer,<NA>,2014490606@qq.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
919,6139d99124ba8b00701a07b1,atlassian,aws-support@strategenics.com.au,AWS-SUPPORT@strategenics.com.au,True,<NA>,<NA>,https://secure.gravatar.com/avatar/6fd0cc8a51c...
65,632113a7cd4a82cf977cafdb,atlassian,<NA>,Adrian Hill,True,<NA>,en_GB,https://secure.gravatar.com/avatar/939dc6b1cfe...
442,557057:fa71e034-c716-4a38-96eb-e3d8f5e106ca,atlassian,<NA>,Adven,False,<NA>,en_US,https://avatar-management--avatars.us-west-2.p...
932,5e5c7ada5a495e0c91a9a2a1,atlassian,<NA>,Agnieszka Klyz,True,<NA>,en_GB,https://secure.gravatar.com/avatar/a12986dd346...
...,...,...,...,...,...,...,...,...
528,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:a131ba...,customer,<NA>,williamvor@gmail.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
266,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:5c32c0...,customer,<NA>,yamiko.kambalametore@intertek.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
27,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:8c9ec0...,customer,<NA>,yasmineazzaoui2@gmail.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
540,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:101b82...,customer,<NA>,yourmail@gmail.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...


In [40]:
# Priorities

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/priority"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.convert_dtypes().infer_objects()
df["id"] = df["id"].astype('Int64')
df = df.sort_values('id')

df.columns = [fnSentenceCase(c) for c in df.columns] 

display(df.dtypes)
display(df)

Status Color    string[python]
Description     string[python]
Icon Url        string[python]
Name            string[python]
Id                       Int64
dtype: object

,Status Color,Description,Icon Url,Name,Id
0,#cc0000,to do,https://strategenics.atlassian.net/images/icon...,Critical,1
1,#ff0000,to do,https://strategenics.atlassian.net/images/icon...,High,2
2,#009900,to do,https://strategenics.atlassian.net/images/icon...,Medium,3
3,#006600,to do,https://strategenics.atlassian.net/images/icon...,Low,4
4,#003300,to do,https://strategenics.atlassian.net/images/icon...,Enhancement,5
